In [1]:
from sympy import *
from scipy.integrate import quad
x, a = symbols('x a')
import math

In [2]:
def jxbar_prime(pe, tb, jxbar, theta, alpha):
    return  pe ** (-alpha * theta) * (pe + tb) ** (-(1 - alpha) * theta) / (
            jxbar * pe ** (-alpha * theta) * (pe + tb) ** (-(1 - alpha) * theta) + (
            1 - jxbar) * (pe + (1- alpha) * tb) ** (-theta)) * jxbar

def j0bar_prime(pe,tb,jxbar, theta, alpha):
    return (pe + tb) ** (-(1 - alpha) * theta) / (
            jxbar * (pe + tb) ** (-(1 - alpha) * theta) + (1 - jxbar) * pe ** (
            -(1 - alpha) * theta)) * jxbar

def CeFF_prime(pe,tb,jxbar_prime, jxbar, sigmastar, theta, alpha):
    epsilonDstar = alpha + (1 - alpha) * sigmastar
    return Ceystar * ((1 - jxbar_prime) / (1 - jxbar)) ** (1 + (1 - sigmastar) / theta) * pe ** (-epsilonDstar)

def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1))

    Bfunvec1_prime = quad(tempFunction, 0, j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction, 0, jxbar_prime, args=(theta, sigmastar))[0]

    return (Bfunvec1_prime, Bfunvec2_prime)

In [7]:
Cex = 0.421601
Ceystar = 26.059874
Cey = 4.598402945
Cem = 1.196110986
jxbar = Cex/(Cex + Ceystar)
alpha = 0.85
theta = 4
## phi = 5
pe = 1.010698
tb = 1.762878
tau = 1.1
sigmastar = 1
sigma = 1
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
varphi = 1

Vg_prime = 38.63009333
Vgstar_prime = 176.5431669

prop = 0.449691508

In [4]:
def g(p, alpha = 0.15):
    return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha

def gprime(p, alpha = 0.15):
    return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))

def k(p):
    return gprime(p) / (g(p)-p*gprime(p))

def gdprime(p, alpha = 0.15):
    return -alpha * p**(alpha - 2) * ((1-alpha)/alpha)**alpha

def Dstar(p, sigmastar):
    return gprime(p) * g(p)**(-sigmastar)

def Dstarprime(p, sigmastar):
    return diff(Dstar(x, sigmastar),x).subs(x,p)

def D1star(p,t,sigmastar):
    return g(p)**(-sigmastar) * gprime(p + t)

def D1starprime(p,t,sigmastar):
    return diff(D1star(x,t,sigmastar),x).subs(x,p)

In [5]:
def jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe) + tb * gprime(pe))**(-theta) * Ceystar
    return num / denum

def j0_new(pe,tb,jxbar,theta,Cex,Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe))**(-theta) * Ceystar
    return num / denum

def Ceystar_new(pe,tb, jxbar_p, jxbar, sigmastar, theta, Ceystar):
    const = gprime(pe) * g(pe)**(-sigmastar) * Ceystar
    frac = ((1-jxbar_p)/ (1-jxbar))**(1+(1-sigmastar)/theta)
    frac = ((1-jxbar_new(pe,tb,jxbar,theta,Cex,Ceystar))/ (1-jxbar))**(1+(1-sigmastar)/theta)
    return const * frac / (g(1)**(-sigmastar) * gprime(1))

def Cey_new(pe,tb, sigmastar, theta, Cey):
    const = gprime(pe + tb) * g(pe+tb)**(-sigma) * Cey
    return const / (g(1)**(-sigma) * gprime(1))

def Cex1_new(pe,tb, j0bar_p, j0bar, sigmastar, Cex):
    const = gprime(pe + tb) * g(pe+tb)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = (j0bar_p / j0bar) ** (1+ (1-sigmastar)/theta)
    return const * frac

def Cex2_new(pe,tb,jxbar_p, j0bar_p, jxbar, sigmastar, theta):
    const = g(pe)**(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = ((1-jxbar)/jxbar)**(sigmastar/theta) * (theta + 1 - sigmastar)/theta
    jterm = 1/ jxbar**(1+(1-sigmastar)/theta)
    B1, B2 = imcomp_betas(j0bar_p, jxbar_p, theta, sigmastar)
    return const * frac * jterm * (B2- B1)
    

def Cem_new(pe, tb, sigma, Cem):
    const = gprime(pe + tb) * g(pe + tb)**(-sigma) / (g(1)**(-sigmastar) * gprime(1)) * Cem
    return const
    
def Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar):
    pterm = g(pe)**(1-sigmastar) *g(1)* Cex / (gprime(1) * g(1)**(1-sigmastar))
    num = (1-j0bar_p)**((theta + 1 - sigmastar)/theta) - (1-jxbar_p)**((theta + 1 - sigmastar) / theta)
    denum = jxbar * (1-jxbar)**((1-sigmastar)/theta)
    return pterm * num / denum

def Vgx1_new(pe, tb, j0_p, jxbar, Cex, theta, sigmastar):
    const = (g(pe+tb) / g(1))**(1-sigmastar) * (j0_p / jxbar)**(1+(1-sigmastar)/theta) * g(1)/gprime(1) * Cex
    return const

def S_new(pe, tb, Cex2_prime, Vgx2_prime):
    const = g(pe+tb) / gprime(pe+tb) * Cex2_prime - Vgx2_prime
    return const

def Vgm_new(pe, tb, Cem):
    return g(pe+tb)**(1-sigma) * Cem * g(1) / (g(1)**(1-sigma) * gprime(1))

def Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime):
    return (1/k(pe+tb)) * (Cey_prime + Cex1_prime + Cex2_prime)

def Lgstar_new(pe,tb,Ceystar_prime, Cem_prime):
    return 1/k(pe) * Ceystar_prime + 1/k(pe+tb) * Cem_prime

def Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime):
    Vgystar_prime = (g(pe) / g(1))**(1-sigmastar) * ((1-jxbar_prime)/(1-jxbar)) ** (1+(1-sigmastar)/theta) * g(1)/ gprime(1) * Ceystar
    return Vgystar_prime + Vgx1_prime + Vgx2_prime

def Vgystar_new(pe,Ceystar, jxbar_prime, jxbar, sigmastar, theta):
    return (g(pe) / g(1))**(1-sigmastar) * ((1-jxbar_prime)/(1-jxbar)) ** (1+(1-sigmastar)/theta) * g(1)/ gprime(1) * Ceystar


In [6]:
jxbar_prime = jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar)
j0_prime = j0_new(pe,tb,jxbar,theta,Cex,Ceystar)
Cey_prime = Cey_new(pe,tb, sigmastar, theta, Cey)
Cex1_prime = Cex1_new(pe,tb, j0_prime, jxbar, sigmastar, Cex)
Cex2_prime = Cex2_new(pe, tb, jxbar_prime, j0_prime, jxbar, sigmastar, theta)
Cem_prime = Cem_new(pe, tb, sigma, Cem)
Ceystar_prime = Ceystar_new(pe,tb, jxbar_prime, jxbar, sigmastar, theta, Ceystar)

Vgx1_prime = Vgx1_new(pe, tb, j0_prime, jxbar, Cex, theta, sigmastar)
Vgx2_prime = Vgx2_new(pe, jxbar_prime, jxbar, j0_prime, Cex, theta, sigmastar)
Vgm_prime = Vgm_new(pe, tb, Cem)
Vgystar_prime = Vgystar_new(pe,Ceystar, jxbar_prime, jxbar, sigmastar, theta)

Vgx = Vgx1_new(1, 0, jxbar, jxbar, Cex, theta, sigmastar)
Vgystar = Vgystar_new(1,Ceystar, jxbar, jxbar, sigmastar, theta)
Vgstar = Vgstar_new(1,jxbar, jxbar, sigmastar, theta, Ceystar, Vgx, 0) 
S = S_new(pe, tb, Cex2_prime, Vgx2_prime)

In [114]:
Qestar_prime = 27.9442294910064

In [115]:
epsilonDstar = (alpha + (1 - alpha) * sigmastar)

In [122]:
((Vgx1_prime + Vgx2_prime) - Vgx) + (Vgystar_prime - Vgystar)

-1.7763568394002505e-14

In [118]:
jxbar_prime/jxbar-1

0.3741799580431471

In [119]:
(Vgystar_prime - Vgystar) / Vgystar

-0.006053545941586292

In [120]:
Vgystar_prime + Vgx1_prime + Vgx2_prime - Vgstar

-2.842170943040401e-14

In [121]:
Vgx + Vgystar

176.54316666666665